In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.environ['PARKIT_STORAGE_PATH'] = 'C:\\Users\\rdpuser\\Desktop\\test'
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [3]:
import random

import parkit as pk

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [4]:
s = pk.Stream('s1')

In [22]:
pk.Dict()

In [6]:
for item in s:
    print(item)

3
3


In [15]:
[d.path for d in pk.directories()]

['default']

In [25]:
[o.path for o in pk.directory().objects(include_hidden = True)]

['default/__8036b115-b2b5-4fcf-9236-925570ba2b93__', 'default/s1']

In [20]:
pk.pool().size

4

In [12]:
s.descriptor

{'databases': [['378b76d3385a9c1f472d5427dc378c8e972369bd',
   {'integerkey': True}],
  ['b9649fdff198a853c6c227c2b6cde956e5d3e142', {'integerkey': True}]],
 'origin': 'a20ff8ad-7cee-468b-96e8-fb8baec7e014',
 'uuid': '15a1b53d-ccb5-409b-a0fb-25fd1b883b9c',
 'anonymous': False,
 'versioned': True,
 'created': '2021-06-09 07:47:56.909081',
 'type': 'parkit.adapters.stream.Stream',
 'metadata': {}}

In [ ]:
d1 = pk.Dict('1/1')
d2 = pk.Dict('1/2')

In [ ]:
d1[1] = 1

In [ ]:
from pynng import Pair0

s1 = Pair0()
s1.listen('tcp://127.0.0.1:54321')
s2 = Pair0()
s2.dial('tcp://127.0.0.1:54321')
s1.send(b'Well hello there')
print(s2.recv())
s1.close()
s2.close()

In [ ]:
from pynng import Pub0

s1 = Pub0()
s1.listen('tcp://127.0.0.1:54321')
try:
    for _ in pk.polling_loop(1):
        s1.send(b'puppy: that is a cute dog')
finally:
    s1.close()

In [ ]:
class CustomDict(pk.Dict):
    
    def get_metadata(self, value):
        return value
    
    def decode_value(self, value, meta):
        value = super().decode_value(value)
        assert meta == value
        return value
    
class CustomArray(pk.Array):
    
    def get_metadata(self, value):
        return value
    
    def decode_value(self, value, meta):
        value = super().decode_value(value)
        assert meta == value
        return value

In [ ]:
class Test():
    
    def random_operation(self):
        if len(self._obj1) > 1e6:
            self._obj1.clear()
            self._obj2.clear()
        random.choice(self._ops)()
        
    def random_value(self):
        return random.randint(0, 1e6)
    
    def compare(self):
        assert len(self._obj1) == len(self._obj2)

In [ ]:
class MutableSequenceTest(Test):
    
    def __init__(self, obj):
        self._limit = 1e2
        self._obj1 = []
        self._obj2 = obj
        self._ops = [
            self.set, self.extend,
            self.append, self.get, self.contains,
            self.maybe_contains, self.count, self.len,
            self.index, self.iterate
        ]
            
    def compare(self):
        super().compare()
        for i, _ in enumerate(self._obj1):
            assert self._obj2[i] == self._obj1[i]
        print(len(self._obj1), len(self._obj2))
        
    def get(self):
        if len(self._obj1):
            i = random.randrange(0, len(self._obj1))
            assert self._obj1[i] == self._obj2[i]
            
    def maybe_contains(self):
        value = self.random_value()
        assert (value in self._obj1) == (value in self._obj2)
        
    def iterate(self):
        if len(self._obj1):
            a = random.randrange(-len(self._obj1), len(self._obj1))
            b = random.randrange(-len(self._obj1), len(self._obj1))
            s1 = self._obj1[a:b]
            s2 = list(self._obj2[a:b])
            assert len(s1) == len(s2)
            for i in range(len(s1)):
                assert s1[i] == s2[i]
            
    def contains(self):
        if len(self._obj1):
            value = random.choice(self._obj1)
            assert value in self._obj2
    
    def index(self):
        if len(self._obj1):
            value = random.choice(self._obj1)
            assert self._obj1.index(value) == self._obj2.index(value)
        
    def count(self):
        if len(self._obj1):
            value = random.choice(self._obj1)
            assert self._obj1.count(value) == self._obj2.count(value)
        
    def len(self):
        assert len(self._obj1) == len(self._obj2)
        
    def append(self):
        value = self.random_value()
        self._obj1.append(value)
        self._obj2.append(value)
        
    def extend(self):
        values = [self.random_value() for _ in range(random.randint(0, self._limit))]
        self._obj1.extend(values)
        self._obj2.extend(values)
        
    def set(self):
        if len(self._obj1):
            i = random.randrange(0, len(self._obj1))
            value = self.random_value()
            self._obj1[i] = value
            self._obj2[i] = value
    
    def clear(self):
        self._obj1.clear()
        self._obj2.clear()

In [ ]:
#a = pk.Array()
a = CustomArray()
test = MutableSequenceTest(a)

In [ ]:
for i in range(100):
    test.random_operation()

In [ ]:
test.compare()

In [ ]:
class MutableMappingTest(Test):
    
    def __init__(self, obj):
        self._limit = 1e2
        self._obj1 = dict()
        self._obj2 = obj
        self._ops = [
            self.set, self.get, 
            self.pop
        ]
        
    def compare(self):
        super().compare()
        for key in self._obj1.keys():
            assert self._obj1[key] == self._obj2[key]
        print(len(self._obj1), len(self._obj2))
                
    def pop(self):
        if len(self._obj1):
            key = random.choice(list(self._obj1.keys()))
            assert self._obj1.pop(key) == self._obj2.pop(key)
            
    def get(self):
        if len(self._obj1):
            key = random.choice(list(self._obj1.keys()))
            assert self._obj1[key] == self._obj2[key]
        
    def set(self):
        key = self.random_value()
        value = self.random_value()
        self._obj1[key] = value
        self._obj2[key] = value
    
    def clear(self):
        self._obj1.clear()
        self._obj2.clear()

In [ ]:
#a = pk.Dict()
a = CustomDict()
test = MutableMappingTest(a)

In [ ]:
for i in range(1000):
    test.random_operation()

In [ ]:
test.compare()

In [ ]:
class MutableQueueTest():
    pass